In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!pip install pyspark

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

In [6]:
totalvotes_df = df.filter("total_votes>=20")
totalvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37480006|R1B4753QMTD8ON|B00U6AE4BK|     632231911|Maxpedition Chubb...|          Sports|          5|           32|         32|   N|                N|Maxpedition Organ...|As a preamble, ha...| 2015-08-31|
|         US|   49868087|R28BVHOEL6GCHZ|B011CCGRX8|     403987913|5 Resistance Loop...|          Sports|          5|    

In [7]:
helpfulvotes_df = totalvotes_df.filter("helpful_votes/total_votes<=.50")
helpfulvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50641517|R19G3LWRFBHDX3|B010MEXZZU|      92869912|Brunswick Tzone D...|          Sports|          1|            4|         42|   N|                Y|            One Star|I can't find the ...| 2015-08-31|
|         US|    1155154| RNGLCHDY6TZZ7|B00Q2LJ4JE|     280867213|12-in-1 Bandana H...|          Sports|          1|    

In [8]:
vinereview_df = helpfulvotes_df.filter(helpfulvotes_df["vine"] == "Y")
vinereview_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51663540|R2P2AVI50P612D|B00VS36SN6|     689268241|Jillian Michaels ...|          Sports|          3|           10|         24|   Y|                N|For all out extre...|I was initially s...| 2015-05-09|
|         US|   51847546| R1N4Z1GW39KVY|B004KOXDWE|     303436672|Rip 60 Fitness DV...|          Sports|          2|    

In [9]:
nonvinereview_df = helpfulvotes_df.filter(helpfulvotes_df["vine"] == "N")
nonvinereview_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50641517|R19G3LWRFBHDX3|B010MEXZZU|      92869912|Brunswick Tzone D...|          Sports|          1|            4|         42|   N|                Y|            One Star|I can't find the ...| 2015-08-31|
|         US|    1155154| RNGLCHDY6TZZ7|B00Q2LJ4JE|     280867213|12-in-1 Bandana H...|          Sports|          1|    

In [14]:
from pyspark.sql.functions import count

vine_count = vinereview_df.count()
print("Total count of Paid Reviews: %f" % vine_count)

nonvine_count = nonvinereview_df.count()
print("Total cound of NON-Paid Reviews: %f" % nonvine_count)

Total count of Paid Reviews: 2.000000
Total cound of NON-Paid Reviews: 6046.000000


In [16]:
star5_vinereview = vinereview_df.filter(nonvinereview_df["star_rating"] == "5")
print("Total number of Paid 5 star reviews: %f " % star5_vinereview.count())

star5_nonvinereview = nonvinereview_df.filter(nonvinereview_df["star_rating"] == "5")
print("Total number of Non-Paid 5 star reviews: %f " % star5_nonvinereview.count())

Total number of Paid 5 star reviews: 0.000000 
Total number of Non-Paid 5 star reviews: 690.000000 


In [17]:
star5_vinepercent = (star5_vinereview.count()/vine_count)
print("Percent of Paid 5 star reviews: %f" % star5_vinepercent)

star5_nonvinepercent = (star5_nonvinereview.count()/ nonvine_count)
print("Percent of Non Paid 5 star reviews: %f" % star5_nonvinepercent)


Percent of Paid 5 star reviews: 0.000000
Percent of Non Paid 5 star reviews: 0.114125
